In [2]:
import cv2 as cv
import numpy as np
from imutils import contours

In [3]:
def scaleImg(image, scale: float):
    width, height = image.shape[1], image.shape[0]
    return cv.resize(image, (int(width * scale), int(height * scale)))

def clipImg(image, max_size, filter=cv.INTER_CUBIC):
    width, height = image.shape[1], image.shape[0]
    max_dim = max(width, height)
    ratio = float(max_size) / max_dim
    return cv.resize(image, (int(width*ratio), int(height*ratio)), interpolation=filter)

In [12]:

orig_img = cv.imread('task6-test2.jpg')
#'task6-3.PNG'
#'task6-test2.jpg'
img = orig_img
img = clipImg(img, 600)

controlls_window_name = 'controlls'
cv.namedWindow(controlls_window_name)


def pass_callback(x):
    pass


total_area = img.shape[0] * img.shape[1]

width = img.shape[1]
height = img.shape[0]

# text params
font = cv.FONT_ITALIC
fontScale = 1.5
fontColor = (0, 255, 0)
thickness = 1


def approx_contour(cntr):
    perimeter = cv.arcLength(cntr, True)
    approx = cv.approxPolyDP(cntr, 0.02 * perimeter, True)
    return approx


def filter_contours(cntrs, tresh=0.002):
    global area_ratio
    filtered = []
    approximate_cntrs = []

    for c in cntrs:
        hull = cv.convexHull(c)

        area = cv.contourArea(hull)
        approx = approx_contour(c)

        if area / total_area > tresh:
            filtered.append(c)
            approximate_cntrs.append(approx)
    return filtered, approximate_cntrs


def vect_len(v):
    v = np.reshape(v, -1)
    return np.sqrt(v[0] ** 2 + v[1] ** 2)


def getPt(x):
    x = np.reshape(x, -1)
    return [x[0], x[1]]


def diff(x, y):
    x = x.ravel()
    y = y.ravel()
    return [x[0] - y[0], x[1] - y[1]]


treshold1 = 100
treshold2 = 100
grades = [0.7, 0.3]


def classify_shape(approximate):
    if len(approximate) == 3:
        shape = "triangle"

    elif len(approximate) == 4:
        (x, y, w, h) = cv.boundingRect(approximate)
        fig_area = cv.contourArea(approximate)
        approx_area = vect_len(diff(approximate[0], approximate[1])) ** 2

        ar = fig_area / approx_area
        shape = "square" if 0.9 <= ar <= 1.2 else "rectangle"
    elif len(approximate) == 5:
        shape = "pentagon"
    elif len(approximate) == 6:
        shape = "hexagon"
    else:
        M = cv.moments(approximate)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        radius = vect_len(approximate[0] - (cX, cY))

        approx_area = np.pi * (radius ** 2)
        fig_area = cv.contourArea(approximate)
        ar = approx_area / fig_area

        if 0.9 <= ar <= 1.2:
            shape = "circle"
        elif 0.5 <= ar <= 2.0:
            shape = "ellipse"
        else:
            shape = "unknown"
    return shape


def classify_area(cntr):
    global total_area, grades, width, height
    _, _, w, h = cv.boundingRect(hull)
    _ = cv.contourArea(cntr)
    ratio = max(w, h) / min(width, height)
    if ratio >= grades[0]:
        return ratio, 'large'
    elif ratio >= grades[1]:
        return ratio, 'medium'
    else:
        return ratio, 'small'


def getContours(img, t1, t2):
    blurred = cv.blur(img, (3, 3))
    edges_canny = cv.Canny(blurred, t1, t2)

    contours, hierarchy = cv.findContours(edges_canny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    contours, approxs = filter_contours(contours)

    total_contour = np.vstack(contours)
    hull = cv.convexHull(total_contour)

    return hull, total_contour


max_ratio = -1
for i in range(10):
    for j in range(10):
        t1 = np.int0(i * 255 / 10)
        t2 = np.int0(j * 255 / 10)

        hull, total_contour = getContours(img, t1, t2)
        area_ratio, size_label = classify_area(hull)

        if area_ratio > max_ratio:
            max_ratio = area_ratio

            treshold1 = t1
            treshold2 = t2
mode = 0
modes_count = 2


def nextMode(x):
    global mode
    mode = (mode + x) % modes_count


def set_treshold1(x):
    global treshold1
    treshold1 = x


def set_treshold2(x):
    global treshold2
    treshold2 = x


#cv.createTrackbar('treshold 1', controlls_window_name, treshold1, 800, set_treshold1)
#cv.createTrackbar('treshold 2', controlls_window_name, treshold2, 800, set_treshold2)

canvas = np.zeros(img.shape, dtype=np.uint8)
overlay = np.zeros(img.shape, dtype=np.uint8)
approx_cntrs = np.zeros(img.shape, dtype=np.uint8)


def flatten(cntrs):
    list_of_pts = []
    for ctr in cntrs:
        list_of_pts += [pt[0] for pt in ctr]
    return np.asarray(list_of_pts)


def approx_contour(cntr):
    perimeter = cv.arcLength(cntr, True)
    approx = cv.approxPolyDP(cntr, 0.02 * perimeter, True)
    return approx


while True:
    canvas[:, :, :] = 0
    overlay[:, :, :] = 0
    approx_cntrs[:, :, :] = 0
    hull, total_contour = getContours(img, treshold1, treshold2)
    area_ratio, size_label = classify_area(hull)
    type = classify_shape(approx_contour(hull))
    if mode == 1:
        cv.drawContours(overlay, total_contour, -1, (50, 255, 0), 1)
    else:
        x, y, w, h = cv.boundingRect(hull)
        cv.putText(overlay, 'area: ' + "{:.1f}".format(area_ratio * 100) + '%',
                   (np.int32(x + w / 2 - 40), np.int32(y + h / 2)), font, 0.6, fontColor, thickness)
        cv.putText(overlay, size_label,
                   (np.int32(x + w / 2 - 40), np.int32(y + h / 2) + 15), font, 0.6, fontColor, thickness)
        cv.putText(overlay, type,
                   (np.int32(x + w / 2 - 40), np.int32(y + h / 2) + 32), font, 0.6, fontColor, thickness)

    cv.drawContours(overlay, [hull], 0, (255, 10, 10))
    cv.imshow('orig', img)
    cv.imshow('task', cv.addWeighted(img, 0.3, overlay, 0.7, 0.0))
    k = cv.waitKey(1) & 0xFF
    if k == ord('q'):
        nextMode(-1)
    if k == ord('e'):
        nextMode(1)
    if k == 27:
        break
cv.destroyAllWindows()